In [19]:
import csv
import re
import numpy as np

param_re = re.compile(r"ch(\d+)=(\-?\d+\.?\d*e?[+\-]?\d*)", re.IGNORECASE)
mc_re    = re.compile(r"mc_iteration=(\d+)", re.IGNORECASE)

def parse_puf_mc_csv(path, n_bits=8):
    """
    path: CSV exported from ADE with Parameters+diff rows
    returns:
      challenges : (n_challenges, n_bits) float
      bits       : (n_chips, n_challenges) int (1 if diff>0 else 0)
    """
    # data[(ch_tuple)][mc] = diff
    data = {}

    with open(path, newline='') as f:
        r = csv.reader(f)
        current_ch = None
        current_mc = None

        for row in r:
            if not row:
                continue

            # Parameters line
            if row[0].startswith("Parameters:"):
                line = row[0]

                # extract ch0..ch7
                vals = {int(m.group(1)): float(m.group(2))
                        for m in param_re.finditer(line)}
                current_ch = tuple(vals[i] for i in range(n_bits))

                # extract mc_iteration
                m = mc_re.search(line)
                current_mc = int(m.group(1)) if m else None

            # Data line with diff
            elif row[1:3] == ['base_setup', 'diff'] and row[3] and current_ch is not None:
                diff_str = row[3]
                # allow engineering suffixes like "27.12m"
                diff_str = diff_str.replace('m', 'e-3').replace('u', 'e-6')
                diff = float(diff_str)

                if current_ch not in data:
                    data[current_ch] = {}
                if current_mc is not None:
                    data[current_ch][current_mc] = diff

    # sort challenges to get deterministic order
    chal_list = sorted(data.keys())
    n_challenges = len(chal_list)

    # infer number of chips (max mc_iteration)
    example = next(iter(data.values()))
    n_chips = max(example.keys())

    challenges = np.array(chal_list, dtype=float)          # (n_challenges, 8)
    diffs = np.zeros((n_chips, n_challenges), dtype=float)

    for j, ch in enumerate(chal_list):
        for mc, d in data[ch].items():
            diffs[mc - 1, j] = d       # mc_iteration is 1‑based

    bits = (diffs > 0).astype(int)
    return challenges, bits


# -------- example usage --------
challenges, bits = parse_puf_mc_csv("data/8b30c.csv")
print("challenges shape:", challenges.shape)   # (256, 8)
print("bits shape:", bits.shape)              # (30, 256)


challenges shape: (256, 8)
bits shape: (30, 256)


In [20]:
import numpy as np

n_chips, n_challenges = bits.shape


def uniformity(outputs_chip):
    return outputs_chip.mean()


def inter_chip_uniqueness(all_bits):
    n_chips, n_ch = all_bits.shape
    dists = []
    for i in range(n_chips - 1):
        xor = all_bits[i] ^ all_bits[i + 1:]
        hd = xor.mean(axis=1)
        dists.append(hd)
    if not dists:
        return np.nan, np.nan, np.array([])
    dists = np.concatenate(dists)
    return dists.mean(), dists.std(), dists


def bit_aliasing(all_bits):
    return all_bits.mean(axis=0)


# ---------- metrics ----------

uniformities = np.apply_along_axis(uniformity, 1, bits)
print("Uniformity per chip:", uniformities)
print("Uniformity mean over chips:", uniformities.mean())
print("Uniformity std over chips :", uniformities.std())

uniq_mean, uniq_std, uniq_all = inter_chip_uniqueness(bits)
print("Inter‑chip HD mean (uniqueness):", uniq_mean)
print("Inter‑chip HD std :", uniq_std)

alias = bit_aliasing(bits)            
print("Bit‑aliasing mean over 256 challenges:", alias.mean())
print("Bit‑aliasing std over 256 challenges :", alias.std())


Uniformity per chip: [0.74609375 0.         0.125      0.40625    1.         1.
 0.38671875 0.         0.8671875  1.         0.0390625  1.
 0.6640625  0.47265625 0.         0.7421875  0.98046875 0.66796875
 0.61328125 0.         0.63671875 0.50390625 1.         0.0078125
 0.45703125 1.         0.48828125 0.0859375  0.70703125 0.08984375]
Uniformity mean over chips: 0.5229166666666667
Uniformity std over chips : 0.36778781453318593
Inter‑chip HD mean (uniqueness): 0.49516882183908045
Inter‑chip HD std : 0.2901077746089451
Bit‑aliasing mean over 256 challenges: 0.5229166666666667
Bit‑aliasing std over 256 challenges : 0.1007135997105985
